Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### logistic regression example with SGD 

In [23]:
# implement a logistic model
batch_size = 128
regularation_param = 0.0015

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed ----------------------------------------1
    # at run time with a training minibatch.
    #  相当于开辟空间
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.                                                                                                       ------------------------------------------2
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.                                                                                  ------------------------------------------3
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ regularation_param * tf.nn.l2_loss(weights)
  
  # Optimizer.                                                                                                       -------------------------------------------4
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.                             --------------------------------------------5
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        #  传递值到tf的命名空间
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 24.299789
Minibatch accuracy: 7.0%
Validation accuracy: 9.1%
Minibatch loss at step 500: 3.484118
Minibatch accuracy: 73.4%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 1.490088
Minibatch accuracy: 82.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.940244
Minibatch accuracy: 86.7%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 0.952981
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 0.777083
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.496503
Minibatch accuracy: 91.4%
Validation accuracy: 81.8%
Test accuracy: 88.8% with regularation_param = 0.001500 


### 1-hidden layer neural network with rectified linear units nn.relu() 

In [32]:
#SGD
batch_size = 128
num_hidden=1024
regularation_param = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  relu = tf.nn.relu(logits)
  logits_hidden = tf.matmul(relu, weights_hidden) + biases_hidden
  #loss = tf.reduce_mean(
   # tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden)) + regularation_param * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)),weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)),weights_hidden)+biases_hidden)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3474.947510
Minibatch accuracy: 5.5%
Validation accuracy: 31.9%
Minibatch loss at step 500: 21.311613
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 1000: 0.903116
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 0.656199
Minibatch accuracy: 85.9%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 0.818116
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.710676
Minibatch accuracy: 85.2%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.510267
Minibatch accuracy: 90.6%
Validation accuracy: 83.7%
Test accuracy: 89.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?只要简单的把训练数据的范围缩小并重复训练就能得到overfitting.

---

### logistic regression example with SGD

In [24]:
# in lofistic regression  overfitting version
batch_size = 128
regularation_param = 0.0015

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed ----------------------------------------1
    # at run time with a training minibatch.
    #  相当于开辟空间
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.                                                                                                       ------------------------------------------2
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.                                                                                  ------------------------------------------3
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+ regularation_param * tf.nn.l2_loss(weights)
  
  # Optimizer.                                                                                                       -------------------------------------------4
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.                             --------------------------------------------5
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # restrict train dataset in a smaller dataset 
        if offset > 10000:
            offset = 0
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        #  传递值到tf的命名空间
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%% with regularation_param = %f " %( accuracy(test_prediction.eval(), test_labels), regularation_param))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 19.324303
Minibatch accuracy: 21.1%
Validation accuracy: 19.9%
Minibatch loss at step 500: 2.059161
Minibatch accuracy: 100.0%
Validation accuracy: 67.9%
Minibatch loss at step 1000: 0.987879
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 1500: 0.484146
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
Minibatch loss at step 2000: 0.237953
Minibatch accuracy: 100.0%
Validation accuracy: 75.1%
Minibatch loss at step 2500: 0.131692
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 3000: 0.081911
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 81.6% with regularation_param = 0.001500 


### 1-hidden layer neural network with rectified linear units nn.relu()

In [25]:
#SGD
batch_size = 128
num_hidden=1024
regularation_param = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  relu = tf.nn.relu(logits)
  logits_hidden = tf.matmul(relu, weights_hidden) + biases_hidden
  #loss = tf.reduce_mean(
   # tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden)) + regularation_param * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)),weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)),weights_hidden)+biases_hidden)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    if offset > 10000:
            offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3475.470215
Minibatch accuracy: 3.9%
Validation accuracy: 34.0%
Minibatch loss at step 500: 20.921240
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 1000: 0.358630
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 1500: 0.199578
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 2000: 0.190236
Minibatch accuracy: 100.0%
Validation accuracy: 73.4%
Minibatch loss at step 2500: 0.184370
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Minibatch loss at step 3000: 0.180916
Minibatch accuracy: 100.0%
Validation accuracy: 71.8%
Test accuracy: 79.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [34]:
#SGD
batch_size = 128
num_hidden=1024
regularation_param = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden]))
  biases = tf.Variable(tf.zeros([num_hidden]))
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  relu = tf.nn.relu(logits)
  temp = tf.nn.dropout(relu, 0.9)
  logits_hidden = tf.matmul(temp, weights_hidden) + biases_hidden
  #loss = tf.reduce_mean(
   # tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden)) + regularation_param * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)),weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
  tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)),weights_hidden)+biases_hidden)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3502.242676
Minibatch accuracy: 18.0%
Validation accuracy: 27.6%
Minibatch loss at step 500: 21.311405
Minibatch accuracy: 80.5%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.901782
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 0.670776
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.839159
Minibatch accuracy: 82.0%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.731820
Minibatch accuracy: 85.9%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.529084
Minibatch accuracy: 91.4%
Validation accuracy: 83.7%
Test accuracy: 89.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [6]:
# without regularation
# test accuracy 96.3%
batch_size =128
regularation_param = 0.002

num_steps = 20001
def compute_logits(data, weightss, biasess, dropout_vals=None):
    temp = data
    if dropout_vals:
        for w,b,d in zip(weightss[:-1], biasess[:-1], dropout_vals[:-1]):
            temp = tf.nn.relu_layer(tf.nn.dropout(temp, d), w, b)
        temp = tf.matmul(temp, weightss[-1]) + biasess[-1]
    else:
        for w,b in zip(weightss[:-1], biasess[:-1]):
            temp = tf.nn.relu_layer(temp, w, b)
        temp = tf.matmul(temp, weightss[-1]) + biasess[-1]
    return temp
    


graph = tf.Graph()
with graph.as_default():
    # input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # variable
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.08, global_step, 200, 0.98)
    weights1 = tf.Variable(tf.truncated_normal([784, 1024], stddev = np.sqrt(2.0/1024)))
    biases1 = tf.Variable(tf.zeros([1024]))
    
    weights2 = tf.Variable(tf.truncated_normal([1024, 800], stddev = np.sqrt(2.0/800)))
    biases2 = tf.Variable(tf.zeros([800]))
    
    weights3 = tf.Variable(tf.truncated_normal([800, 512], stddev = np.sqrt(2.0/512)))
    biases3 = tf.Variable(tf.zeros([512]))
    
    weights4 = tf.Variable(tf.truncated_normal([512, 256], stddev = np.sqrt(2.0/256)))
    biases4 = tf.Variable(tf.zeros([256]))
    
    weights5 = tf.Variable(tf.truncated_normal([256, 10], stddev = np.sqrt(2.0/10)))
    biases5 = tf.Variable(tf.zeros([10]))
    
    #training
    logits = compute_logits(tf_train_dataset, [weights1, weights2, weights3,weights4, weights5], [biases1,biases2,biases3,biases4, biases5], \
                            dropout_vals=(1.0,0.95,0.95,0.95,1.0))
    #temp = tf.nn.relu_layer(tf_train_dataset, weights1, biases1)
    #temp = tf.nn.relu_layer(tf.nn.dropout(temp, 0.95), weights2, biases2)
    #temp = tf.nn.relu_layer(tf.nn.dropout(temp, 0.95), weights3, biases3)
    #logits = tf.matmul(temp, weights4) + biases4
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))  + regularation_param * (
        tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4) + tf.nn.l2_loss(weights5))
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(compute_logits(tf_valid_dataset, [weights1, weights2, weights3, weights4, weights5], \
                                                    [biases1, biases2, biases3, biases4,biases5]))
    test_prediction = tf.nn.softmax(compute_logits(tf_test_dataset, [weights1, weights2, weights3, weights4,weights5],\
                                                   [biases1, biases2, biases3, biases4,biases5]))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        #  传递值到tf的命名空间
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%% " %( accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 11.299002
Minibatch accuracy: 7.8%
Validation accuracy: 18.7%
Minibatch loss at step 500: 4.983230
Minibatch accuracy: 82.8%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 4.353046
Minibatch accuracy: 85.2%
Validation accuracy: 87.1%
Minibatch loss at step 1500: 3.723177
Minibatch accuracy: 89.1%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 3.456311
Minibatch accuracy: 85.2%
Validation accuracy: 88.2%
Minibatch loss at step 2500: 3.029123
Minibatch accuracy: 86.7%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 2.535067
Minibatch accuracy: 96.1%
Validation accuracy: 88.9%
Minibatch loss at step 3500: 2.398166
Minibatch accuracy: 89.1%
Validation accuracy: 88.8%
Minibatch loss at step 4000: 2.355282
Minibatch accuracy: 85.9%
Validation accuracy: 88.8%
Minibatch loss at step 4500: 2.083345
Minibatch accuracy: 88.3%
Validation accuracy: 89.2%
Minibatch loss at step 5000: 1.971526
Minibatch accuracy: 87.5%
Validation accurac